# Introduction to `GeoGraph` - 4 March 2021

**Recent progress:**
- **WS1 - ongoing:** habitat classification
  - First results using XGBoost (See video below)
  - Additional work: Infrared, UNet
- **WS2 - completed:** fragmentation baseline finished (Show summary notebook)
- **WS3&4 - ongoing:** building GeoGraph platform
  - Completed: loading & accessing, habitats, basic visualisation, node identification
  - Ongoing: timeline, advanced visualisation, metrics

In [ ]:
from IPython.display import Video, HTML
display(HTML(("Video: each of the top 4 panels is an RGB image - so that there are 12 input"
         " bands in total. The last 5 years of the video is the test set"
         " the rest of it was in the training set")))
Video("exalted-dust-26_joint_val.mp4", width=600)

---

## 0. Setup and Loading package

In [ ]:
from src.data_loading import jasmin
from src.models import geograph, visualisation
from src.constants import PREFERRED_CRS

---

## 1. Loading Data

In [ ]:
# Loading Polesia data set from shared Jasmin folder
gdf = jasmin.get_polesia_data()

# Looking at south-west corner of data
# Choosen because are of wilderness as discussed
minx, miny, maxx, maxy = gdf.total_bounds
square_len = 25000

gdf = gdf.cx[ minx:minx+square_len  ,  miny:miny+square_len]

print("Number of patches in region of interest:", len(gdf))
gdf.head(5)

---

## 2. Creating `GeoGraph`

In [ ]:
# Building the main graph structure

graph = geograph.GeoGraph(gdf, 
                          crs=PREFERRED_CRS, 
                          columns_to_rename={"Eunis_name":"class_label","AREA":"area"})

---

## 3. Creating Habitats

In [ ]:
# First selecting the classes that make up our habitat
# We chose all classes with 'pine' in the name.
pine_classes = [label for label in graph.df.class_label.unique() if 'pine' in label]
pine_classes

In [ ]:
# Proposed distance from email: mobile (<100m), semi mobile (<25m) and sessile (<5m)
graph.add_habitat('Sessile (5m) - pine', 
                  max_travel_distance=5, 
                  valid_classes=pine_classes)

graph.add_habitat('Semi mobile (25m) - pine', 
                  max_travel_distance=25, 
                  valid_classes=pine_classes)

graph.add_habitat('Mobile (500m) - pine',
                  max_travel_distance=500, 
                  valid_classes=pine_classes)

---

## 4. Interactive Graph

In [ ]:
viewer = visualisation.GeoGraphViewer()
viewer.add_graph(graph, name='Polesia data')
viewer.add_widgets()
viewer.add_hover_widget()
viewer

In [ ]:
viewer.log

### Things to consider in GeoGraphViewer
- How `max_travel_distance` affects the number of components in habitat

## 5. Next Steps

See questions doc...

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True